# Install and import libraries

In [ ]:

%pip install "numpy~=1.26.0" "ml-dtypes~=0.4.0" "tensorflow-intel==2.18.0" "mediapipe~=0.10.0" --force-reinstall

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization # For CNN
# from tensorflow.keras.layers import LSTM, GRU # Optional: if you want to try other recurrent layers

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pathlib import Path

from constants import (
    DATA_INPUT_PATH,
    MODEL_PATH,        # This will be for the Keras model file
    METADATA_PATH,     # This will be for the scaler and other metadata pickle file
    PREDICTED_LANDMARK_NAMES, # Defines the target landmark columns
    NUM_EMG_SENSORS,
    WINDOW_SIZE,
    WINDOW_STEP
    # Add any other relevant constants like NUM_PREDICTED_LANDMARKS if defined,
    # or derive NUM_PREDICTED_LANDMARKS from PREDICTED_LANDMARK_NAMES later
)

## Convert .csv(s) to dataframes and concatenate

In [ ]:
# Cell 5: Convert .csv(s) to dataframes and concatenate

from pathlib import Path
import pandas as pd

# --- Constants Loading (Ensure these are available from Cell 3) ---
# from constants import DATA_INPUT_PATH, NUM_EMG_SENSORS, PREDICTED_LANDMARK_NAMES
# ---

dfs = []

# Set your input path from constants.py
data_input_dir = Path(DATA_INPUT_PATH)

# Find CSV files
files = list(data_input_dir.glob("*.csv"))
print(f"Looking for CSV files in: {data_input_dir.resolve()}")
print(f"Found CSVs: {files}")

if not files:
    raise FileNotFoundError(f"No CSV files found in {data_input_dir.resolve()}. Please check DATA_INPUT_PATH in constants.py and your data directory.")

# Read all files
for file_path in files:
    try:
        df_temp = pd.read_csv(file_path)
        # Optional: Add a check for necessary columns *before* appending
        # required_for_load = [f"s{i}_norm" for i in range(1, NUM_EMG_SENSORS + 1)] + ["quat_w"] + PREDICTED_LANDMARK_NAMES[:1]
        # if not all(col in df_temp.columns for col in required_for_load):
        #     print(f"Warning: Skipping {file_path}, missing essential columns like sX_norm or landmarks.")
        #     continue
        dfs.append(df_temp)
        print(f"Successfully loaded {file_path}, shape: {df_temp.shape}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        continue # Skip to next file if one is problematic

if not dfs:
    raise ValueError("No DataFrames were successfully loaded. Check CSV files for issues.")

# Concatenate into one DataFrame
df_concat = pd.concat(dfs, axis=0, ignore_index=True)
print(f"Shape after concatenation of all CSVs: {df_concat.shape}")

# --- Define Feature and Target Columns ---

# <<< CHANGE: Use _norm suffix for EMG columns as these are the input features now >>>
emg_cols = [f"s{i}_norm" for i in range(1, NUM_EMG_SENSORS + 1)] # e.g., ['s1_norm', ..., 's8_norm']
imu_cols = ["quat_w", "quatx", "quaty", "quatz"]

# Rename quaternion columns ONLY IF loading data that used old names (quat_x etc.)
# Your newer data likely already has quatx, quaty, quatz based on collection.py
cols_to_rename = {}
if 'quat_x' in df_concat.columns and 'quatx' not in df_concat.columns: cols_to_rename['quat_x'] = 'quatx'
if 'quat_y' in df_concat.columns and 'quaty' not in df_concat.columns: cols_to_rename['quat_y'] = 'quaty'
if 'quat_z' in df_concat.columns and 'quatz' not in df_concat.columns: cols_to_rename['quat_z'] = 'quatz'
if cols_to_rename:
    print(f"Found old quaternion column names. Renaming: {cols_to_rename}")
    df_concat.rename(columns=cols_to_rename, inplace=True)

feature_columns = emg_cols + imu_cols # Feature columns now use pre-normalized EMG + quats
target_landmark_columns = PREDICTED_LANDMARK_NAMES # Target columns remain the landmark coordinates

# --- Validate Columns ---
print(f"\nVerifying required columns...")
print(f"Expected Feature Columns ({len(feature_columns)}): {feature_columns[:5]}...")
print(f"Expected Target Columns ({len(target_landmark_columns)}): {target_landmark_columns[:5]}...")

available_columns_list = df_concat.columns.tolist()

missing_features = [col for col in feature_columns if col not in available_columns_list]
if missing_features:
    raise ValueError(f"Critical Error: Missing required input feature columns in the loaded data: {missing_features}. "
                     f"These columns (incl. sX_norm) should be generated by collection.py. Available columns: {available_columns_list}")

missing_targets = [col for col in target_landmark_columns if col not in available_columns_list]
if missing_targets:
    raise ValueError(f"Critical Error: Missing required target landmark columns in the loaded data: {missing_targets}. "
                     f"Check constants.PREDICTED_LANDMARK_NAMES vs CSV headers. Available columns: {available_columns_list}")

print("All required feature and target columns found.")

# --- Select Columns and Clean Data ---
# Keep only the columns needed for windowing + potentially id/gesture_id for context
columns_to_keep = []
if 'timestamp' in available_columns_list: columns_to_keep.append('timestamp')
if 'gesture_id' in available_columns_list: columns_to_keep.append('gesture_id')
columns_to_keep.extend(feature_columns)
columns_to_keep.extend(target_landmark_columns)
final_columns_to_keep = sorted(list(set(columns_to_keep)), key=columns_to_keep.index) # Unique, ordered

df_processed = df_concat[final_columns_to_keep].copy()
print(f"\nShape after selecting relevant columns ({len(final_columns_to_keep)} columns): {df_processed.shape}")

# Drop rows with any NaNs in the essential feature/target columns
# (Check only feature/target columns for NaNs that would break training)
essential_cols = feature_columns + target_landmark_columns
shape_before_na_drop = df_processed.shape
df_processed.dropna(subset=essential_cols, inplace=True)
print(f"Shape after dropping rows with NaN in features/targets: {df_processed.shape}. (Dropped {shape_before_na_drop[0] - df_processed.shape[0]} rows)")

if df_processed.empty:
    raise ValueError("DataFrame is empty after processing and NaN removal. Check raw data.")

# Drop duplicate rows (can be intensive, consider sampling if dataset is huge)
shape_before_duplicates = df_processed.shape
df_processed.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate rows: {df_processed.shape}. (Dropped {shape_before_duplicates[0] - df_processed.shape[0]} rows)")

if df_processed.empty:
    raise ValueError("DataFrame is empty after dropping duplicates.")

print(f"\n--- Final Processed DataFrame Head (df_processed used for windowing) ---")
print(df_processed.head())
print(f"\nUsing Feature columns ({len(feature_columns)}): {feature_columns}")
print(f"Using Target landmark columns ({len(target_landmark_columns)}): {target_landmark_columns[:5]}... (first 5)")

In [ ]:
# calibration_mean and calibration_std derived here are for an old Z-score normalization method. We are now using StandardScaler fitted on X_train. Kept for archival purposes.
# --- OLD Calculate Calibration Stats from 'Rest' Data (Z-Score) ---
"""
import numpy as np
from constants import NUM_EMG_SENSORS # Make sure NUM_EMG_SENSORS is defined in constants.py (should be 8)

print("Calculating calibration statistics from 'Rest' data...")

# Select only the 'Rest' data (gesture_id == 0)
rest_df = df[df['gesture_id'] == 0].copy()

# Select only the EMG columns (s1 to s8)
emg_columns = [f"s{i}" for i in range(1, NUM_EMG_SENSORS + 1)]

# Select only IMU columns (quat)
imu_columns = ["quat_w","quatx","quaty","quatz"]

# Get landmark data
landmark_columns = [c for c in df.columns if c.endswith(("_x","_y","_z"))]
print(f"Landmark columns are: {landmark_columns}")

rest_emg_data = rest_df[emg_columns].values # Get as numpy array

if len(rest_emg_data) > 0:
    # Calculate mean and std dev for each EMG channel (column-wise)
    calibration_mean = np.mean(rest_emg_data, axis=0)
    calibration_std = np.std(rest_emg_data, axis=0)

    # Avoid division by zero: set std dev to 1 if it's 0 or very close to 0
    calibration_std[calibration_std < 1e-6] = 1.0

    print(f"Calculated Calibration Mean (shape {calibration_mean.shape}):\n{np.round(calibration_mean, 2)}")
    print(f"Calculated Calibration StdDev (shape {calibration_std.shape}):\n{np.round(calibration_std, 2)}")
else:
    print("ERROR: No 'Rest' data found to calculate calibration statistics!")
    # Handle this error appropriately - maybe exit or use default values
    # Using default values for now, but this indicates a data problem
    calibration_mean = np.zeros(NUM_EMG_SENSORS)
    calibration_std = np.ones(NUM_EMG_SENSORS)
    print("Using default calibration stats (mean=0, std=1). CHECK YOUR DATA.")

# Ensure these variables are available for the next cell
# (They will be if run in the same kernel session)
# --- END OF NEW CELL ---
"""

In [ ]:
# This cell should directly follow Cell 5 (Data Loading/Processing)
# It uses 'df_processed', 'feature_columns', and 'target_landmark_columns' from Cell 5.
# It also uses WINDOW_SIZE and WINDOW_STEP from constants.py (imported in Cell 3).

print(f"\n--- Starting Windowing Process ---")
print(f"Using WINDOW_SIZE: {WINDOW_SIZE}, WINDOW_STEP: {WINDOW_STEP}")
print(f"Input features for windowing ({len(feature_columns)}): {feature_columns}")
print(f"Target features for windowing ({len(target_landmark_columns)}): {target_landmark_columns[:5]}... (first 5)")

X_sequences = []
Y_targets = []

# Ensure df_processed is not empty before windowing
if df_processed.shape[0] < WINDOW_SIZE:
    raise ValueError(
        f"Not enough data rows ({df_processed.shape[0]}) in df_processed to create even one window of size {WINDOW_SIZE}."
        " Check your data or filtering steps."
    )

# Create sequences
# Loop from the first possible start index up to a point where a full window can still be formed.
for start_index in range(0, df_processed.shape[0] - WINDOW_SIZE + 1, WINDOW_STEP):
    end_index = start_index + WINDOW_SIZE
    
    # Extract input sequence (X): EMG + IMU data for the current window
    # .values converts the DataFrame slice to a NumPy array
    input_window_data = df_processed[feature_columns].iloc[start_index:end_index].values
    X_sequences.append(input_window_data)
    
    # Extract target (Y): Landmark coordinates at the END of the current window
    # .values converts the Series to a NumPy array
    target_values_at_window_end = df_processed[target_landmark_columns].iloc[end_index - 1].values
    Y_targets.append(target_values_at_window_end)

if not X_sequences:
    raise ValueError("No sequences were generated. This might happen if df_processed has fewer rows than WINDOW_SIZE.")

# Convert lists of sequences/targets to NumPy arrays
X_win = np.array(X_sequences)
Y = np.array(Y_targets) # Renaming Y_targets to Y for consistency with subsequent cells

print(f"\n--- Windowing Complete ---")
print(f"Shape of input sequences (X_win): {X_win.shape}")  # Expected: (num_samples, WINDOW_SIZE, num_input_features)
print(f"Shape of target landmarks (Y): {Y.shape}")      # Expected: (num_samples, num_target_landmark_coords)

# Basic validation of shapes
# num_input_features should be NUM_EMG_SENSORS (8) + number of IMU channels (4) = 12
expected_num_input_features = NUM_EMG_SENSORS + len(imu_cols) 
if X_win.shape[2] != expected_num_input_features:
    print(f"Warning: X_win.shape[2] (number of features) is {X_win.shape[2]}, but expected {expected_num_input_features}.")

# num_target_values should be the total number of landmark coordinates defined in PREDICTED_LANDMARK_NAMES
expected_num_target_values = len(PREDICTED_LANDMARK_NAMES)
if Y.shape[1] != expected_num_target_values:
    print(f"Warning: Y.shape[1] (number of target values) is {Y.shape[1]}, but expected {expected_num_target_values} based on PREDICTED_LANDMARK_NAMES.")

## Split Data, Scale Input Features not required

In [ ]:
# Cell 9: Split Data (No Scaling Needed)

from sklearn.model_selection import train_test_split
# NO StandardScaler import needed here

# Make sure X_win and Y are defined from Cell 7 (Windowing)
if 'X_win' not in locals() or 'Y' not in locals():
    raise NameError("X_win or Y is not defined. Ensure Cell 7 (Windowing) was run successfully.")

print(f"\n--- Starting Data Split ---")
print(f"Original X_win shape: {X_win.shape}, Y shape: {Y.shape}")

# 1) Split into Training + Validation vs. Test sets (e.g., 80% train+val, 20% test)
X_trainval, X_test, Y_trainval, Y_test = train_test_split(
    X_win, Y, test_size=0.20, random_state=42, stratify=None
)

# 2) Split Training + Validation into actual Training vs. Validation sets (e.g., 80% * 0.75 = 60% train, 80% * 0.25 = 20% val)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_trainval, Y_trainval, test_size=0.25, random_state=42, stratify=None
)

print("\nShapes after splitting:")
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape:   {X_val.shape}, Y_val shape:   {Y_val.shape}")
print(f"X_test shape:  {X_test.shape}, Y_test shape:  {Y_test.shape}")

# <<< SCALING BLOCK REMOVED >>>
# Input data (X_train, X_val, X_test) now uses pre-normalized EMG columns (sX_norm)
# and raw quaternion data. We will feed this directly to the model.
# Target data (Y_train, Y_val, Y_test) contains normalized landmark coordinates.

# Define constants needed for model input shape based on the data
num_train_samples, window_size_const, num_input_features_const = X_train.shape
print(f"\nData shapes confirmed for model: window_size={window_size_const}, num_input_features={num_input_features_const}")

print(f"--- Data Split Complete (No Scaling Applied in this Notebook) ---")

## Build, compile & train landmark regression model

In [ ]:
# Cell 11: Build, compile & train landmark regression model

# Imports should be at the top (Cell 3)
# Ensure necessary variables from Cell 9 are available:
# X_train, Y_train, X_val, Y_val, X_test, Y_test
# window_size_const, num_input_features_const
if 'X_train' not in locals(): raise NameError("Training data (X_train) not found. Ensure Cell 9 ran.")
if 'window_size_const' not in locals(): raise NameError("window_size_const not found. Ensure Cell 9 ran.")
if 'num_input_features_const' not in locals(): raise NameError("num_input_features_const not found. Ensure Cell 9 ran.")

print(f"\n--- Building and Training Model ---")

n_outputs = Y_train.shape[1] # Number of target landmark coordinates
print(f"Model output layer size (n_outputs): {n_outputs}")
print(f"Model input shape: (window_size={window_size_const}, features={num_input_features_const})")

# Define the Keras Sequential model (architecture remains the same)
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(window_size_const, num_input_features_const), name="conv1d_1"),
    MaxPooling1D(pool_size=2, name="maxpool1d_1"),
    BatchNormalization(name="batchnorm_1"),
    Conv1D(filters=64, kernel_size=3, activation='relu', name="conv1d_2"),
    MaxPooling1D(pool_size=2, name="maxpool1d_2"),
    BatchNormalization(name="batchnorm_2"),
    Flatten(name="flatten"),
    Dense(units=128, activation='relu', name="dense_1"),
    Dropout(rate=0.3, name="dropout_1"),
    Dense(units=64, activation='relu', name="dense_2"),
    Dense(units=n_outputs, activation="linear", name="output_landmarks") # Linear activation for regression
])

model.summary()

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

model_path_obj = Path(MODEL_PATH) # Should point to .keras file now
model_path_obj.parent.mkdir(parents=True, exist_ok=True)
print(f"Models checkpoints will be saved to: {model_path_obj.resolve()}")

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_PATH, save_best_only=True, monitor='val_loss', verbose=1)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

print("\nStarting model training (using pre-normalized EMG input)...")
history = model.fit(
    X_train, Y_train,                       # <<< USE UNSCALED X_train >>>
    validation_data=(X_val, Y_val),       # <<< USE UNSCALED X_val >>>
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

print("\nModel training finished.")

# Load the best model saved by ModelCheckpoint
print(f"Loading the best model saved at: {MODEL_PATH}")
if not model_path_obj.exists():
    print(f"Warning: Best model file {MODEL_PATH} not found. Using model from last epoch.")
    # Ensure fallback model is compiled if needed
    if not getattr(model, '_is_compiled', True):
        print("Compiling model from last epoch...")
        model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
else:
    try:
        # Load model (using compile=False as robust workaround for potential loading issues)
        model = tf.keras.models.load_model(MODEL_PATH, compile=False)
        print("Best model structure/weights loaded successfully (compile=False).")
        # Re-compile the loaded model
        model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
        print("Best model re-compiled successfully.")
    except Exception as e:
        print(f"ERROR loading/compiling best model: {e}. Using model from last epoch.")
        # Compile fallback model if necessary
        if not getattr(model, '_is_compiled', True):
             print("Compiling model from last epoch (fallback)...")
             model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

# Evaluate the loaded model on the UNscaled test set
print("\nEvaluating final model on the test set...")
if model is not None:
    test_loss, test_mae = model.evaluate(X_test, Y_test, verbose=0) # <<< USE UNSCALED X_test >>>
    print(f"Test Set Performance: Loss (MSE) = {test_loss:.4f}, Metric (MAE) = {test_mae:.4f}")
else:
    print("ERROR: Model object is None, cannot evaluate.")
print(f"--- Model Building and Training Complete ---")

## Save the model and the scalar + feature list

In [ ]:
# Cell 13: Save Metadata Artifacts (No Scaler)

print(f"\n--- Saving Metadata Artifacts (No Scaler) ---")

metadata_path_obj = Path(METADATA_PATH)
metadata_path_obj.parent.mkdir(parents=True, exist_ok=True)

# Ensure required variables are available
if 'PREDICTED_LANDMARK_NAMES' not in locals(): raise NameError("PREDICTED_LANDMARK_NAMES not found.")
if 'feature_columns' not in locals(): raise NameError("feature_columns (using sX_norm) not found.") # Should use sX_norm now

# <<< CHANGE: Removed 'input_scaler' from metadata_to_save >>>
metadata_to_save = {
    "predicted_landmark_names_ordered": PREDICTED_LANDMARK_NAMES,
    "input_feature_names_ordered": feature_columns # Contains sX_norm + quat names
}

try:
    with open(METADATA_PATH, "wb") as f:
        pickle.dump(metadata_to_save, f)
    print(f"Successfully saved metadata to: {metadata_path_obj.resolve()}")
    print(f"Metadata now contains: {list(metadata_to_save.keys())}")
except Exception as e:
    print(f"ERROR saving metadata to {METADATA_PATH}: {e}")

print(f"--- Metadata Saving Complete ---")

# Reminder: Keras model (.keras) was saved in Cell 11.

In [ ]:
import os
from constants import DATA_INPUT_PATH, MODEL_INPUT_PATH, MODEL_PATH, METADATA_PATH

print("DATA_INPUT_PATH ->", DATA_INPUT_PATH)
print("  contains:", os.listdir(DATA_INPUT_PATH))
print()
print("MODEL_INPUT_PATH ->", MODEL_INPUT_PATH)
print("  contains:", os.listdir(MODEL_INPUT_PATH))
print()
print("MODEL_PATH       ->", MODEL_PATH, "exists?", os.path.exists(MODEL_PATH))
print("METADATA_PATH    ->", METADATA_PATH, "exists?", os.path.exists(METADATA_PATH))


In [ ]:
import constants
print(constants.__file__)


## DEMO: test the new model for accuracy

In [ ]:
# Cell 17: DEMO: test the new model for accuracy

import matplotlib.pyplot as plt # Should already be imported

print(f"\n--- Evaluating Trained Regression Model with Loaded Artifacts ---")

# 1. Load the trained regression Keras model
model_path_obj = Path(MODEL_PATH) # Should point to .keras file
if not model_path_obj.exists(): raise FileNotFoundError(...)
print(f"Loading regression model from: {model_path_obj.resolve()}")
try:
    # Load using compile=False for robustness
    loaded_regression_model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    print("Regression model structure/weights loaded successfully (compile=False).")
    # Re-compile
    loaded_regression_model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
    print("Model re-compiled successfully.")
    loaded_regression_model.summary()
except Exception as e:
    raise IOError(f"Error loading/compiling Keras model from {MODEL_PATH}: {e}")


# 2. Load the metadata (landmark names, feature names - NO SCALER)
metadata_path_obj = Path(METADATA_PATH)
if not metadata_path_obj.exists(): raise FileNotFoundError(...)
print(f"Loading metadata from: {metadata_path_obj.resolve()}")
try:
    with open(METADATA_PATH, "rb") as f: loaded_metadata = pickle.load(f)

    # <<< CHANGE: Do not load/expect 'input_scaler' >>>
    loaded_pred_landmark_names = loaded_metadata.get("predicted_landmark_names_ordered")
    loaded_input_feature_names = loaded_metadata.get("input_feature_names_ordered") # Should contain sX_norm

    # <<< CHANGE: Check required keys excluding scaler >>>
    if loaded_pred_landmark_names is None or loaded_input_feature_names is None:
        raise ValueError(f"Metadata file {METADATA_PATH} is missing required keys. Found: {list(loaded_metadata.keys())}")

    print("Metadata loaded successfully.")
    print(f"  Number of predicted landmark names: {len(loaded_pred_landmark_names)}")
    print(f"  Number of input feature names: {len(loaded_input_feature_names)}")
    print(f"  Input feature names example: {loaded_input_feature_names[:5]}...") # Verify it shows _norm

except Exception as e:
    raise IOError(f"Error loading or parsing metadata from {METADATA_PATH}: {e}")


# 3. Use some data from X_test and Y_test (defined in Cell 9) for demonstration
if 'X_test' not in locals() or 'Y_test' not in locals(): raise NameError(...) # X_test contains sX_norm

num_samples_to_demo = min(5, X_test.shape[0])
if num_samples_to_demo == 0:
    print("No samples in X_test to demonstrate.")
else:
    # <<< CHANGE: Use X_test directly as it contains pre-normalized EMG >>>
    sample_X_for_prediction = X_test[:num_samples_to_demo]
    sample_Y_true = Y_test[:num_samples_to_demo]
    print(f"\nUsing first {num_samples_to_demo} samples from X_test (which contains sX_norm) for prediction...")
    print(f"Shape of sample_X_for_prediction: {sample_X_for_prediction.shape}")

    # <<< REMOVED: Step 4 (applying scaler) is no longer needed >>>

    # 5. Make predictions with the loaded model
    predicted_Y_values = loaded_regression_model.predict(sample_X_for_prediction)
    print(f"Shape of predicted_Y_values: {predicted_Y_values.shape}")


    # 6. Visualize/Compare some predictions against true values
    num_coords_to_plot_per_sample = min(3, len(loaded_pred_landmark_names))

    for i in range(num_samples_to_demo):
        print(f"\n--- Sample {i+1} ---")
        plt.figure(figsize=(12, num_coords_to_plot_per_sample * 2))
        plt.suptitle(f"Sample {i+1}: Predicted vs. True Landmark Coordinates", fontsize=14)

        for coord_idx in range(num_coords_to_plot_per_sample):
            true_val = sample_Y_true[i, coord_idx]
            pred_val = predicted_Y_values[i, coord_idx]
            landmark_name = loaded_pred_landmark_names[coord_idx]

            print(f"  Landmark: {landmark_name}, True: {true_val:.4f}, Predicted: {pred_val:.4f}, Diff: {abs(true_val - pred_val):.4f}")

            ax = plt.subplot(num_coords_to_plot_per_sample, 1, coord_idx + 1)
            ax.bar(['True Value', 'Predicted Value'], [true_val, pred_val], color=['skyblue', 'salmon'])
            ax.set_title(f"{landmark_name}")
            ax.set_ylabel("Coordinate Value")
            ax.grid(axis='y', linestyle='--')

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

    # Optional: Re-evaluate on the full X_test (unscaled) if desired
    if 'X_test' in locals() and 'Y_test' in locals():
         print("\nRe-evaluating the loaded model on the full test set (using pre-normalized EMG)...")
         # Ensure model is compiled before evaluate
         if not getattr(loaded_regression_model, '_is_compiled', True):
             loaded_regression_model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
         full_test_loss, full_test_mae = loaded_regression_model.evaluate(X_test, Y_test, verbose=0) # Use X_test
         print(f"  Full Test Set Performance: Loss (MSE) = {full_test_loss:.4f}, Metric (MAE) = {full_test_mae:.4f}")

print(f"\n--- Evaluation with Loaded Artifacts Complete ---")

# Define num_input_features_const if not defined (needed for old Cell 17 reshape - now removed)
if 'num_input_features_const' not in locals() and 'loaded_input_feature_names' in locals():
     num_input_features_const = len(loaded_input_feature_names)
     print(f"Note: 'num_input_features_const' was derived from loaded metadata as {num_input_features_const}")